# Setup

## Install packages and download data sets

In [ ]:
%%bash
pip install kaggle -U scikit-learn 
cp /content/drive/MyDrive/pyism/kaggle.json /content
cp /content/drive/MyDrive/pyism/{kaggle_data,utils}.py /content

In [ ]:
import kaggle_data
kaggle_api, _ = kaggle_data.activate_kaggle("bluebook-for-bulldozers", kaggle_credential_path="/content/kaggle.json")

kaggle.json Copied!
---------------------------------------------
List of Data Files of bluebook-for-bulldozers: 
Valid.csv
TrainAndValid.zip
Machine_Appendix.csv
Valid.zip
Valid.7z
random_forest_benchmark_test.csv
Train.7z
Train.zip
Data Dictionary.xlsx
median_benchmark.csv
TrainAndValid.7z
TrainAndValid.csv
ValidSolution.csv
Test.csv


In [ ]:
# kaggle_data.download_file_kaggle(kaggle_api, "bluebook-for-bulldozers", True)
file_name = ['Data Dictionary.xlsx','Train.zip', 'Valid.zip','ValidSolution.csv']
kaggle_data.download_file_kaggle(kaggle_api, "bluebook-for-bulldozers", True, *file_name)

100%|██████████| 10.8k/10.8k [00:00<00:00, 8.20MB/s]


Error ocurred while unzipping file: Data Dictionary.xlsx.zip.\Details:
[Errno 2] No such file or directory: 'Data Dictionary.xlsx.zip'


100%|██████████| 9.28M/9.28M [00:00<00:00, 22.6MB/s]


100%|██████████| 297k/297k [00:00<00:00, 50.9MB/s]

100%|██████████| 316k/316k [00:00<00:00, 56.7MB/s]


Error ocurred while unzipping file: ValidSolution.csv.zip.\Details:
[Errno 2] No such file or directory: 'ValidSolution.csv.zip'


In [ ]:
!rm *.zip

## Import necessary packages

In [ ]:
import numpy as np
import pandas as pd
import re 
import matplotlib.pyplot as plt
import utils
from sklearn.base import TransformerMixin, BaseEstimator
%matplotlib inline

## Read in Data

In [ ]:
# data set
train = pd.read_csv("Train.csv", low_memory=False, parse_dates=['saledate'])
valid = pd.read_csv("Valid.csv", low_memory=False, parse_dates=['saledate'])
# read in target for the valid set
valid_sol = pd.read_csv("ValidSolution.csv").drop(columns='Usage')
valid = valid.merge(valid_sol, on='SalesID')

In [ ]:
train.head()

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,fiModelDesc,fiBaseModel,fiSecondaryDesc,fiModelSeries,fiModelDescriptor,ProductSize,fiProductClassDesc,state,ProductGroup,ProductGroupDesc,Drive_System,Enclosure,Forks,Pad_Type,Ride_Control,Stick,Transmission,Turbocharged,Blade_Extension,Blade_Width,Enclosure_Type,Engine_Horsepower,Hydraulics,Pushblock,Ripper,Scarifier,Tip_Control,Tire_Size,Coupler,Coupler_System,Grouser_Tracks,Hydraulics_Flow,Track_Type,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1139246,66000,999089,3157,121,3.0,2004,68.0,Low,2006-11-16,521D,521,D,NaN,NaN,NaN,Wheel Loader - 110.0 to 120.0 Horsepower,Alabama,WL,Wheel Loader,NaN,EROPS w AC,None or Unspecified,NaN,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2 Valve,NaN,NaN,NaN,NaN,None or Unspecified,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
1,1139248,57000,117657,77,121,3.0,1996,4640.0,Low,2004-03-26,950FII,950,F,II,NaN,Medium,Wheel Loader - 150.0 to 175.0 Horsepower,North Carolina,WL,Wheel Loader,NaN,EROPS w AC,None or Unspecified,NaN,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2 Valve,NaN,NaN,NaN,NaN,23.5,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
2,1139249,10000,434808,7009,121,3.0,2001,2838.0,High,2004-02-26,226,226,NaN,NaN,NaN,NaN,Skid Steer Loader - 1351.0 to 1601.0 Lb Operating Capacity,New York,SSL,Skid Steer Loaders,NaN,OROPS,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Auxiliary,NaN,NaN,NaN,NaN,NaN,None or Unspecified,None or Unspecified,None or Unspecified,Standard,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1139251,38500,1026470,332,121,3.0,2001,3486.0,High,2011-05-19,PC120-6E,PC120,NaN,-6E,NaN,Small,"Hydraulic Excavator, Track - 12.0 to 14.0 Metric Tons",Texas,TEX,Track Excavators,NaN,EROPS w AC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2 Valve,NaN,NaN,NaN,NaN,NaN,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1139253,11000,1057373,17311,121,3.0,2007,722.0,Medium,2009-07-23,S175,S175,NaN,NaN,NaN,NaN,Skid Steer Loader - 1601.0 to 1751.0 Lb Operating Capacity,New York,SSL,Skid Steer Loaders,NaN,EROPS,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Auxiliary,NaN,NaN,NaN,NaN,NaN,None or Unspecified,None or Unspecified,None or Unspecified,Standard,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
drop_cols = ['ProductGroup', 'fiModelDesc', 'SalesID']
train.drop(columns=drop_cols, inplace=True)
valid.drop(columns=drop_cols, inplace=True)

In [ ]:
train.shape, valid.shape

((401125, 50), (11573, 50))

In [ ]:
# read in the description
pd.options.display.max_colwidth = 300
data_dict = pd.read_excel('Data%20Dictionary.xlsx', index_col='Variable').drop(columns=['Unnamed: 2']).T
data_dict.head()

Variable,SalesID,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,Saledate,Saleprice,fiModelDesc,fiBaseModel,fiSecondaryDesc,fiModelSeries,fiModelDescriptor,ProductSize,ProductClassDesc,State,ProductGroup,ProductGroupDesc,Drive_System,Enclosure,Forks,Pad_Type,Ride_Control,Stick,Transmission,Turbocharged,Blade_Extension,Blade_Width,Enclosure_Type,Engine_Horsepower,Hydraulics,Pushblock,Ripper,Scarifier,Tip_control,Tire_Size,Coupler,Coupler_System,Grouser_Tracks,Hydraulics_Flow,Track_Type,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
Description,unique identifier of a particular sale of a machine at auction,identifier for a particular machine; machines may have multiple sales,identifier for a unique machine model (i.e. fiModelDesc),source of the sale record; some sources are more diligent about reporting attributes of the machine than others. Note that a particular datasource may report on multiple auctioneerIDs.,"identifier of a particular auctioneer, i.e. company that sold the machine at auction. Not the same as datasource.",year of manufacturer of the Machine,current usage of the machine in hours at time of sale (saledate); null or 0 means no hours have been reported for that sale,"value (low, medium, high) calculated comparing this particular Machine-Sale hours to average usage for the fiBaseModel; e.g. 'Low' means this machine has less hours given it's lifespan relative to average of fiBaseModel.",time of sale,cost of sale in USD,Description of a unique machine model (see ModelID); concatenation of fiBaseModel & fiSecondaryDesc & fiModelSeries & fiModelDescriptor,disaggregation of fiModelDesc,disaggregation of fiModelDesc,disaggregation of fiModelDesc,disaggregation of fiModelDesc,Don't know what this is,description of 2nd level hierarchical grouping (below ProductGroup) of fiModelDesc,US State in which sale occurred,identifier for top-level hierarchical grouping of fiModelDesc,description of top-level hierarchical grouping of fiModelDesc,machine configuration; typcially describes whether 2 or 4 wheel drive,machine configuration - does machine have an enclosed cab or not,machine configuration - attachment used for lifting,machine configuration - type of treads a crawler machine uses,machine configuration - optional feature on loaders to make the ride smoother,machine configuration - type of control,machine configuration - describes type of transmission; typically automatic or manual,machine configuration - engine naturally aspirated or turbocharged,machine configuration - extension of standard blade,machine configuration - width of blade,machine configuration - does machine have an enclosed cab or not,machine configuration - engine horsepower rating,machine configuration - type of hydraulics,machine configuration - option,machine configuration - implement attached to machine to till soil,machine configuration - implement attached to machine to condition soil,machine configuration - type of blade control,machine configuration - size of primary tires,machine configuration - type of implement interface,machine configuration - type of implement interface,machine configuration - describes ground contact interface,machine configuration - normal or high flow hydraulic system,machine configuration - type of treads a crawler machine uses,machine configuration - width of crawler treads,machine configuration - length of machine digging implement,machine configuration - attachment used for grabbing,machine configuration - can adjust the operator control configuration to suit the user,machine configuration - type of treads a crawler machine uses,machine configuration - optional interface used to add a backhoe attachment,machine configuration - describes type of blade,machine configuration - describes operator control configuration,"machine configuration - differential type, typically locking or st

## Inspect the training data set

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401125 entries, 0 to 401124
Data columns (total 50 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   SalePrice                 401125 non-null  int64         
 1   MachineID                 401125 non-null  int64         
 2   ModelID                   401125 non-null  int64         
 3   datasource                401125 non-null  int64         
 4   auctioneerID              380989 non-null  float64       
 5   YearMade                  401125 non-null  int64         
 6   MachineHoursCurrentMeter  142765 non-null  float64       
 7   UsageBand                 69639 non-null   object        
 8   saledate                  401125 non-null  datetime64[ns]
 9   fiBaseModel               401125 non-null  object        
 10  fiSecondaryDesc           263934 non-null  object        
 11  fiModelSeries             56908 non-null   object        
 12  fi

In [ ]:
train.nunique().sort_values(ascending=False)

MachineID                   341027
MachineHoursCurrentMeter     15152
ModelID                       5218
saledate                      3919
fiBaseModel                   1950
SalePrice                      899
fiSecondaryDesc                175
fiModelDescriptor              139
fiModelSeries                  122
fiProductClassDesc              74
YearMade                        72
state                           53
auctioneerID                    30
Stick_Length                    29
Undercarriage_Pad_Width         19
Tire_Size                       17
Hydraulics                      12
Blade_Type                      10
Transmission                     8
Travel_Controls                  7
ProductGroupDesc                 6
Blade_Width                      6
Enclosure                        6
ProductSize                      6
datasource                       5
Steering_Controls                5
Differential_Type                4
Ripper                           4
Pad_Type            

In [ ]:
train.isna().sum(axis=0).sort_values(ascending=False)

Blade_Extension             375906
Pushblock                   375906
Tip_Control                 375906
Engine_Horsepower           375906
Enclosure_Type              375906
Blade_Width                 375906
Scarifier                   375895
Hydraulics_Flow             357763
Grouser_Tracks              357763
Coupler_System              357667
fiModelSeries               344217
Steering_Controls           331756
Differential_Type           331714
UsageBand                   331486
fiModelDescriptor           329206
Backhoe_Mounting            322453
Stick                       321991
Pad_Type                    321991
Turbocharged                321991
Blade_Type                  321292
Travel_Controls             321291
Tire_Size                   306407
Track_Type                  301972
Grouser_Type                301972
Stick_Length                301907
Pattern_Changer             301907
Thumb                       301837
Undercarriage_Pad_Width     301253
Ripper              

# Data Pre-processing

In [ ]:
trn = train.copy()
val = valid.copy()
trn.shape, val.shape

((401125, 50), (11573, 50))

## Special Variable

### Datetime

In [ ]:
from typing import Iterable


class DatetimeEnricher(BaseEstimator,TransformerMixin):
    def __init__(self, date_col, date_attrs: Iterable, sort_date=False):
        self.date_col = date_col    # date columns
        self.sort_date = sort_date  # whether to sort date 
        self.date_attrs = date_attrs # date attributes to extract

    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        x = X.copy()
        if self.sort_date: x.sort_values(by=[self.date_col], inplace=True)
        for attr in self.date_attrs: 
            x['sale_'+attr] = getattr(x[self.date_col].dt, attr)
        x.drop(columns=[self.date_col],inplace=True)

        return x

In [ ]:
date_attrs = ['day','year','month','quarter','week','dayofweek',
              'dayofyear','days_in_month','is_leap_year', 
              'is_month_end','is_month_start','is_quarter_end',
              'is_quarter_start','is_year_end','is_year_start','weekofyear'] 

de = DatetimeEnricher(date_col='saledate', date_attrs=date_attrs)

trn = de.fit_transform(trn)
val = de.transform(val)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.


In [ ]:
trn.shape, val.shape

((401125, 65), (11573, 65))

### YearMade

In [ ]:
def clean_YearMade(X):
    assert 'YearMade' in X.columns
    X['YearMade_outlier'] = X.YearMade==1000
    X.YearMade.replace(1000, 1900, inplace=True)
    return X

In [ ]:
trn = clean_YearMade(trn)
val = clean_YearMade(val)

In [ ]:
trn.shape, val.shape

((401125, 66), (11573, 66))

### ID columns

In [ ]:
class IDColumnProcessor(BaseEstimator, TransformerMixin):
    """
    This custom Transformer maps ID the their frequencies.
    """
    def __init__(self, id_cols):
        self.id_cols = id_cols

    def fit(self, X, y=None):
        self.mappings = dict()
        for col in self.id_cols:
            self.mappings[col] = X[col].value_counts(dropna=False)
        return self    

    def transform(self, X, y=None):
        x = X.copy()
        self.new_feat_names = np.array(self.id_cols, dtype='object') + "_freq"
        for old, new in zip(self.id_cols, self.new_feat_names):
            x[new] = x[old].map(self.mappings[old])
        return x

In [ ]:
id_cols = ['MachineID', 'ModelID', 'datasource', 'auctioneerID']
idp = IDColumnProcessor(id_cols)
trn = idp.fit_transform(trn)
val = idp.transform(val)

## Categorical Variable

### Target Encoding
https://maxhalford.github.io/blog/target-encoding/


In [ ]:
from pandas.core.common import random_state
class BasicTargetEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, cols, target, smooth_factor=500, bootstrap=None, random_state=None,dropna=True) -> None:
        self.cols = cols
        self.target = target
        self.smooth_factor = smooth_factor
        self.bootstrap = bootstrap
        self.random_state = random_state
        self.dropna = dropna
        self.mappings = dict()
    
    def fit(self, X):
        if not isinstance(X, pd.DataFrame):
            raise ValueError("X must be an instance of pandas.DataFrame")

        x = X.copy() if self.bootstrap is None \
                     else X.sample(frac=self.bootstrap, replace=True, random_state=self.random_state)

        x['y'] = y = self.target

        m = self.smooth_factor
        for col in self.cols:
            aggs = x.groupby(col, dropna=self.dropna)['y'].agg(['mean', 'count'])
            x_bar, n = aggs['mean'], aggs['count']
            # miu = (x_bar * n) + (m * y.mean()) / (n + m)
            self.mappings[col] = ((x_bar * n) + (m * y.mean())) / (n + m)
    
        return self
    
    def transform(self, X, y=None):
        x = X.copy()
        for col in self.cols:
            x[col] = x[col].map(self.mappings[col])

        return x
                    

In [ ]:
# targetenc_cols = cat_cols[(7 < cardinalities) & (cardinalities < 55)]
cat_cols = trn.select_dtypes(['object', 'category']).columns
cardinalities = trn[cat_cols].nunique()
targetenc_cols = cat_cols[cardinalities > 7]
targetenc_cols

Index(['fiBaseModel', 'fiSecondaryDesc', 'fiModelSeries', 'fiModelDescriptor',
       'fiProductClassDesc', 'state', 'Transmission', 'Hydraulics',
       'Tire_Size', 'Undercarriage_Pad_Width', 'Stick_Length', 'Blade_Type'],
      dtype='object')

In [ ]:
%%time 
# Use YearMade as Target to encode
bte = BasicTargetEncoder(targetenc_cols, trn.SalePrice, 
                         bootstrap=.75, random_state=0, 
                         dropna=False, smooth_factor=5000)

trn = bte.fit_transform(trn)
val = bte.transform(val)

CPU times: user 2.43 s, sys: 57.2 ms, total: 2.48 s
Wall time: 2.49 s


In [ ]:
pd.set_option("display.precision", 3)
trn[targetenc_cols].head()

,fiBaseModel,fiSecondaryDesc,fiModelSeries,fiModelDescriptor,fiProductClassDesc,state,Transmission,Hydraulics,Tire_Size,Undercarriage_Pad_Width,Stick_Length,Blade_Type
0,31222.114,29588.032,29982.580,27772.639,31773.872,33313.484,31042.426,35817.371,34580.162,29622.257,29617.562,29840.916
1,39450.299,37724.674,42546.874,27772.639,38060.105,31758.813,31042.426,35817.371,39847.812,29622.257,29617.562,29840.916
2,28071.596,23746.362,29982.580,27772.639,18653.004,28070.840,31042.426,25991.436,28439.949,29622.257,29617.562,29840.916
3,30990.389,23746.362,31109.384,27772.639,31116.869,32954.501,31042.426,35817.371,28439.949,29622.257,29617.562,29840.916
4,28080.058,23746.362,29982.580,27772.639,18998.898,28070.840,31042.426,25991.436,28439.949,29622.257,29617.562,29840.916


In [ ]:
trn.shape, val.shape

((401125, 70), (11573, 70))

### One-hot Encoding

In [ ]:
class Dummier(BaseEstimator, TransformerMixin):
    def __init__(self, cols=None) -> None:
        self.cols = cols
    
    def fit(self, X):
        return self
    
    def transform(self, X, y=None):
        return pd.get_dummies(X, dummy_na=True, drop_first=True, columns=self.cols)
                    

In [ ]:
dumb = Dummier()
trn = dumb.fit_transform(trn)
val = dumb.fit_transform(val)

In [ ]:
trn.shape, val.shape

((401125, 141), (11573, 134))

In [ ]:
trn, val = trn.align(val, 'inner', axis=1)

## Missing Values

### Imputing

In [ ]:
from utils import impute
trn, nas = impute(trn)
val, _ = impute(val, in_nas=nas)
trn['MachineHoursCurrentMeter_na'] = trn['MachineHoursCurrentMeter'].isna()
val['MachineHoursCurrentMeter_na'] = val['MachineHoursCurrentMeter'].isna()
trn.shape, val.shape

((401125, 140), (11573, 142))

# Experiment

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
# yh - yhat: predicted values
def rmsle(y, yh):
    return np.sqrt(((np.log1p(yh) - np.log1p(y))**2).mean())

In [ ]:
default = {'max_features': 0.5, 'max_samples': 0.5,
           'min_samples_leaf': 3, 'min_samples_split': 5,
           'n_jobs': -1}
random_forest = RandomForestRegressor(30, **default)

In [ ]:
trn, val = trn.align(val, 'inner', axis=1)
drop_cols = ['SalePrice']
xtrn, ytrn = trn.drop(columns=drop_cols), trn['SalePrice']
xval, yval = val.drop(columns=drop_cols), val['SalePrice']

In [ ]:
xtrn.shape, xval.shape

((401125, 138), (11573, 138))

In [ ]:
model = random_forest
model.fit(xtrn.values, ytrn.values)

RandomForestRegressor(max_features=0.5, max_samples=0.5, min_samples_leaf=3,
                      min_samples_split=5, n_estimators=30, n_jobs=-1)

In [ ]:
rmsle(yval, model.predict(xval.values))

0.24919084676539133

In [ ]:
utils.feature_importances(model, xtrn.columns, .95)

,importance,rank,cummulative
features,,,
fiBaseModel,0.236,1,0.236
YearMade,0.175,2,0.411
fiProductClassDesc,0.166,3,0.578
sale_year,0.072,4,0.649
Enclosure_EROPS w AC,0.066,5,0.715
fiSecondaryDesc,0.063,6,0.779
ModelID,0.032,7,0.810
ModelID_freq,0.021,8,0.832
MachineID,0.012,9,0.844


# Bring it all together

## Setup

In [ ]:
xtrn, ytrn = train.drop(columns=['SalePrice']), train['SalePrice']
xval, yval = valid.drop(columns=['SalePrice']), valid['SalePrice']
xtrn = clean_YearMade(xtrn)
xval = clean_YearMade(xval)
xtrn.shape, xval.shape

((401125, 50), (11573, 50))

In [ ]:
date_col = 'saledate'
date_col, id_cols, targetenc_cols

('saledate',
 ['MachineID', 'ModelID', 'datasource', 'auctioneerID'],
 Index(['fiBaseModel', 'fiSecondaryDesc', 'fiModelSeries', 'fiModelDescriptor',
        'fiProductClassDesc', 'state', 'Transmission', 'Hydraulics',
        'Tire_Size', 'Undercarriage_Pad_Width', 'Stick_Length', 'Blade_Type'],
       dtype='object'))

In [ ]:
process_steps = [
    ('datetime', DatetimeEnricher(date_col, date_attrs=date_attrs)),
    ('id', IDColumnProcessor(id_cols)),
    ('target_enc', BasicTargetEncoder(targetenc_cols, target=ytrn, bootstrap=.75, random_state=0, dropna=False, smooth_factor=5000)),
    ('dummies', Dummier()),
]

## Pipeline

In [ ]:
from sklearn.pipeline import Pipeline 
process_pipe = Pipeline(steps=process_steps)
# na_pipe = Pipeline(steps=na_steps)

In [ ]:
%%time
xtrn_ = process_pipe.fit_transform(xtrn)
xval_ = process_pipe.transform(xval)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.


CPU times: user 5.11 s, sys: 224 ms, total: 5.34 s
Wall time: 5.32 s


In [ ]:
from utils import impute
xtrn_, nas = impute(xtrn_)
xval_, _ = impute(xval_, in_nas=nas)
xtrn_['MachineHoursCurrentMeter_na'] = xtrn_['MachineHoursCurrentMeter'].isna()
xval_['MachineHoursCurrentMeter_na'] = xval_['MachineHoursCurrentMeter'].isna()

In [ ]:
# different numbers of columns
xtrn_, xval_ = xtrn_.align(xval_, 'inner', axis=1)
xtrn_.shape, xval_.shape

((401125, 138), (11573, 138))

In [ ]:
xtrn_.to_feather("xtrn.feather")
xval_.to_feather("xval.feather")
ytrn.to_frame().to_feather('ytrn.feather')
yval.to_frame().to_feather('yval.feather')

In [ ]:
!mv /content/*feather /content/drive/MyDrive/Colab\ Notebooks/projects/bulldozers/data/